# Allocation of CCGs into intervention and control groups

Note - set seed for random allocation to ensure repeatability

In [1]:

# Set dates of baseline and follow-up periods
d4 = '2019-03-01' # month after end of follow-up period
d3 = '2018-09-01' # follow-up start
d2 = '2018-07-01' # month after end of baseline period
d1 = '2018-01-01' # baseline start


# Import dataset from BigQuery
import pandas as pd
import numpy as np
GBQ_PROJECT_ID = '620265099307'

q = '''SELECT * FROM ebmdatalab.measures.ccg_data_lpzomnibus
WHERE EXTRACT (YEAR from month)  >= 2018
'''
df1 = pd.read_gbq(q, GBQ_PROJECT_ID, dialect='standard',verbose=False)

        ## note: parsing dates is quite memory-intensive, make sure not too many programmes running

df1["month"] = pd.to_datetime(df1.month)

df1.head() # this gives the first few rows of data

,pct_id,month,numerator,denominator,calc_value,percentile
0,03Y,2018-06-01,8061.86989,104.165,77.395189,0.000000
1,04V,2018-06-01,57374.25724,393.317,145.872813,0.500000
2,01V,2018-06-01,38228.77713,125.030,305.756835,1.000000
3,02Q,2018-06-01,18145.94579,117.489,154.448040,0.582474
4,07G,2018-06-01,29043.80496,176.896,164.185764,0.664948


In [2]:
### filter out the baseline and follow-up periods
import datetime

conditions = [
    (df1['month']  >= d4), # after follow-up period
    (df1['month']  >= d3), # follow-up
    (df1['month']  >= d2), # mid
    (df1['month']  >= d1), # baseline
    (df1['month']  < d1)] # before

choices = ['after', 'follow-up', 'mid', 'baseline','before']
df1['period'] = np.select(conditions, choices, default='0')

df1.head()

,pct_id,month,numerator,denominator,calc_value,percentile,period
0,03Y,2018-06-01,8061.86989,104.165,77.395189,0.000000,baseline
1,04V,2018-06-01,57374.25724,393.317,145.872813,0.500000,baseline
2,01V,2018-06-01,38228.77713,125.030,305.756835,1.000000,baseline
3,02Q,2018-06-01,18145.94579,117.489,154.448040,0.582474,baseline
4,07G,2018-06-01,29043.80496,176.896,164.185764,0.664948,baseline


In [3]:
### aggregate data over 6-month baseline

# take columns of interest from df
df2 = df1[["pct_id","period", "month", "numerator","denominator"]]

# Perform groupby aggregation
agg_6m = df2.groupby(["pct_id","period"]).sum() 

### calculate aggregated measure values
agg_6m["calc_value"] = agg_6m.numerator / agg_6m.denominator

agg_6m = agg_6m.reset_index()
agg_6m = agg_6m.loc[agg_6m.period=="baseline"].rename(columns={"calc_value":"baseline"}).drop("period",axis=1)
agg_6m.head()

,pct_id,numerator,denominator,baseline
0,00C,108261.10945,649.675,166.638872
1,00D,254840.09124,1752.237,145.436999
2,00J,317390.42671,1557.677,203.758820
3,00K,247479.68728,1782.485,138.839703
4,00L,238929.64140,1951.199,122.452729


In [4]:
### select the worst ~50 to be pre-screened
#(Also exclude CCGs 99P, 99Q and 08H as per exclusion criteria)

df3 = agg_6m.copy()
df3.loc[(df3.pct_id!="08H")&(df3.pct_id !="99P")&(df3.pct_id !="99Q")].sort_values(by="baseline", ascending=False).head(50).reset_index()


,index,pct_id,numerator,denominator,baseline
0,25,01V,233995.21962,749.704,312.116808
1,27,01X,361479.54430,1185.007,305.044227
2,190,99K,297555.54012,1022.388,291.039742
3,151,09P,311660.13623,1130.895,275.587156
4,21,01J,272497.81256,989.946,275.265330
5,152,09W,487557.09178,1795.837,271.492954
6,18,01F,211759.20528,787.334,268.957273
7,189,99J,769106.67373,2953.688,260.388597
8,80,05G,340137.10538,1308.840,259.876765
9,182,99A,828848.20620,3196.121,259.329420



### The selected CCGs are pre-screened for joint medicines optimisation teams

In [5]:
# import joint team information
team = pd.read_csv('joint_teams.csv')

#give each team a proxy id
team2 = pd.DataFrame(team.groupby("joint_team")["ccg_id"].agg(["count","max"])).reset_index().rename(columns={"max":"joint_id"})
team = team.merge(team2, on="joint_team")
team.head()

,ccg_id,joint_team,count,joint_id
0,99E,"""Basildon, Brentwood and Thurrock MMT""",1,99E
1,99K,Brighton and HWLH MMT,1,99K
2,09F,EHS and HR CCGs MMT,2,09P
3,09P,EHS and HR CCGs MMT,2,09P
4,03E,Harrogate Shared Services MMT,1,03E


In [6]:
# merge aggregated prescribing data with joint team information
j1 = agg_6m.merge(team, left_on="pct_id",right_on="ccg_id", how="left")
j1.loc[j1.ccg_id.isnull(),["joint_id"]] = j1.pct_id
j1 = j1.drop("ccg_id", axis=1)
j1.head()

,pct_id,numerator,denominator,baseline,joint_team,count,joint_id
0,00C,108261.10945,649.675,166.638872,NaN,NaN,00C
1,00D,254840.09124,1752.237,145.436999,NaN,NaN,00D
2,00J,317390.42671,1557.677,203.758820,North of England CSU (NECS),1.0,00J
3,00K,247479.68728,1782.485,138.839703,NaN,NaN,00K
4,00L,238929.64140,1951.199,122.452729,NaN,NaN,00L


In [7]:
# group CCG data up to joint teams
j2 = j1.groupby("joint_id")["numerator","denominator"].sum().reset_index()
j2["baseline"] = j2.numerator / j2.denominator
j2.head()

,joint_id,numerator,denominator,baseline
0,00C,108261.10945,649.675,166.638872
1,00D,254840.09124,1752.237,145.436999
2,00J,317390.42671,1557.677,203.758820
3,00K,247479.68728,1782.485,138.839703
4,00L,238929.64140,1951.199,122.452729


In [8]:
### calculate percentile for each ccg(now screened for joint working) for spend during baseline period 
# and select the worst 40 to be randomised

j3 = j2.copy()
j3["baseline_ranking"] = j3["baseline"].rank(method='min', pct=True)

top40 = j3.loc[(j3.joint_id!="08H")&(j3.joint_id !="99P")&(j3.joint_id !="99Q")].sort_values(by="baseline_ranking", ascending=False).head(40).reset_index(drop=True)
top40

,joint_id,numerator,denominator,baseline,baseline_ranking
0,01X,361479.54430,1185.007,305.044227,1.000000
1,99K,297555.54012,1022.388,291.039742,0.994709
2,01J,272497.81256,989.946,275.265330,0.989418
3,01V,457415.61157,1680.072,272.259529,0.984127
4,09W,487557.09178,1795.837,271.492954,0.978836
5,01F,211759.20528,787.334,268.957273,0.973545
6,99J,769106.67373,2953.688,260.388597,0.968254
7,05G,340137.10538,1308.840,259.876765,0.962963
8,99A,828848.20620,3196.121,259.329420,0.957672
9,11A,844705.12882,3376.395,250.179594,0.952381


In [9]:
top40.describe()

,numerator,denominator,baseline,baseline_ranking
count,40.000000,40.000000,40.000000,40.000000
mean,404539.759662,1814.334700,223.749036,0.890608
std,207453.129851,907.986977,33.911384,0.066143
min,123669.200780,678.013000,178.288868,0.783069
25%,274889.740273,1162.331250,197.203361,0.834656
50%,345460.292115,1559.266000,216.032410,0.891534
75%,494593.841580,2160.147500,247.246846,0.948413
max,871509.120010,4482.042000,305.044227,1.000000


In [10]:
### allocate bottom CCGs to intervention and control groups 

# set seeds for random number generation to ensure repeatable
# seed1 = 321

df5 = top40.copy()
import random as rd

#np.random.seed(seed1)
df5['rand_num'] = np.random.rand(len(df5))
df5["allocation_ranking"] = df5.rand_num.rank()

df5["allocation_code"]= df5.allocation_ranking.mod(2)

#create final allocation groups
df5['allocation'] = np.where(df5['allocation_code']==0,'con','I')

print (df5.loc[df5.allocation=="I"].joint_id.count(), 'CCGs have been assigned to the intervention group,')
print ("with an average spend of £",round(df5.loc[df5.allocation=="I"].baseline.mean(),0), "per 1000. SD:",round(df5.loc[df5.allocation=="I"].baseline.std(),0))
print (df5.loc[df5.allocation=="con"].joint_id.count(), 'CCGs have been assigned to the control group,')
print ("with an average spend of £",round(df5.loc[df5.allocation=="con"].baseline.mean(),0), "per 1000. SD:",round(df5.loc[df5.allocation=="con"].baseline.std(),0))

#df5.loc[df5.allocation == "I"].to_csv('allocated_ccgs.csv')

20 CCGs have been assigned to the intervention group,
with an average spend of £ 233.0 per 1000. SD: 40.0
20 CCGs have been assigned to the control group,
with an average spend of £ 215.0 per 1000. SD: 25.0


In [11]:
### import CCG names for CCGs allocated to intervention group
q = '''
SELECT
  code,
  name
FROM
  ebmdatalab.hscic.ccgs
WHERE org_type = "CCG"
'''

ccg = pd.io.gbq.read_gbq(q, GBQ_PROJECT_ID, dialect='standard',verbose=False)

ccg.head()
dfm = df5.loc[df5.allocation == "I"].merge(ccg, how='left', left_on='joint_id',right_on='code')

#also join back to joint team info and show whether the joint team has any other CCGs in the eligilbe group.
dfm[["joint_id","name"]].merge(team2, on="joint_id", how="left").sort_values(by="joint_team").rename(columns={"count":"CCGs_included"})

,joint_id,name,joint_team,CCGs_included
9,99E,NHS BASILDON AND BRENTWOOD CCG,"""Basildon, Brentwood and Thurrock MMT""",1.0
1,99K,NHS HIGH WEALD LEWES HAVENS CCG,Brighton and HWLH MMT,1.0
15,03J,NHS NORTH KIRKLEES CCG,North Kirklees Shared Service,3.0
16,08Y,NHS WEST LONDON CCG,North West London CSU,1.0
12,00J,NHS NORTH DURHAM CCG,North of England CSU (NECS),1.0
3,01V,NHS SOUTHPORT AND FORMBY CCG,South Sefton & Southport and Formby CCG MMT,2.0
10,06D,NHS WYRE FOREST CCG,Worcestershire MMT,2.0
0,01X,NHS ST HELENS CCG,NaN,NaN
2,01J,NHS KNOWSLEY CCG,NaN,NaN
4,09W,NHS MEDWAY CCG,NaN,NaN


### Calculate baseline stats for whole population, to use to give context in power calculation

In [12]:
j3["baseline"].describe(percentiles = [.1, .25, .5, .75, .8,.85, .9])

count    189.000000
mean     154.205435
std       46.667971
min       79.955639
10%      103.708413
25%      118.214294
50%      150.439042
75%      173.738830
80%      182.084205
85%      202.321241
90%      222.014348
max      305.044227
Name: baseline, dtype: float64